In [1]:
import bittensor as bt
import storage
from storage.validator.database import *
from storage.validator.bonding import *
from storage.shared.utils import get_redis_password
pw = get_redis_password()


In [2]:
import redis.asyncio as redis
database= r = redis.StrictRedis(db=3, password=pw)
await r.keys()

[b'stats:5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWsot1xiPAPVVHj1',
 b'hotkey:5EZCtyfQrDUQqRKYDdq1eaKSB9Ugkr3GxMb8XadewMRMxdev',
 b'stats:5Gpjcz44ViQJzjA14xAgMSZwTLeULeJdXhjKMV6JM3XrGkUw',
 b'hotkey:5E4yZGgFSYvMn9gG9U6ciHZgXdPcRxmbX27mbTsz7yRspKiy',
 b'hotkey:5C82pPChStvLQhLVWxbQSQ52diLvYXxezqP9jhFWSRkHSFon',
 b'stats:5CaFuijc2ucdoWhkjLaYgnzYrpv62KGt1fWWtUxhFHXPA3KK',
 b'stats:5E4yZGgFSYvMn9gG9U6ciHZgXdPcRxmbX27mbTsz7yRspKiy',
 b'stats:5EuzvsFVopm2fQ8SLsPsvQdqe4CatZbvrYscxXbrPZsNYmPX',
 b'stats:5FxGggBG4sc62S4fkDHgsTiPj4fJtpbbed84Yfm1wWtNNZHC',
 b'hotkey:5CaFuijc2ucdoWhkjLaYgnzYrpv62KGt1fWWtUxhFHXPA3KK',
 b'hotkey:5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWsot1xiPAPVVHj1',
 b'hotkey:5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2',
 b'hotkey:5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH',
 b'stats:5CDTTsvLNyrKX9W9DF2CsvR7jpXaue8SUSntPcem1Psq9eb9',
 b'hotkey:5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz',
 b'stats:5DwMX98sJAaAdb6Hr18H69kr4Py1EQXUHcqDoKnLfZqM617N',
 b'hotkey:5EuzvsFVopm2fQ8SLsPsvQ

In [3]:
ncap = await get_network_capacity(r)
ncap

21990232555520

In [4]:
size_bytes = await get_redis_db_size(r)
size_bytes / (1024 * 1024) # MB

0.17199134826660156

In [5]:
all_hashes = await get_all_hashes_in_database(r)
len(all_hashes)

80

In [6]:
challenge_hashes = await get_all_challenge_hashes(r)
len(challenge_hashes)

80

In [7]:
stats = await get_miner_statistics(r)
len(stats)

20

In [8]:
stats

{'5EuzvsFVopm2fQ8SLsPsvQdqe4CatZbvrYscxXbrPZsNYmPX': {'store_attempts': '263',
  'store_successes': '260',
  'challenge_successes': '248',
  'challenge_attempts': '351',
  'retrieve_successes': '53',
  'retrieve_attempts': '77',
  'total_successes': '841',
  'tier': 'Bronze',
  'storage_limit': '1099511627776'},
 '5HapVLypxEsZWvfwd12jUXdp9W1pXAoSbQVzirC6AUDNYBii': {'store_attempts': '0',
  'store_successes': '0',
  'challenge_successes': '0',
  'challenge_attempts': '0',
  'retrieve_successes': '0',
  'retrieve_attempts': '0',
  'total_successes': '0',
  'tier': 'Bronze',
  'storage_limit': '1099511627776'},
 '5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz': {'store_attempts': '273',
  'store_successes': '271',
  'challenge_successes': '267',
  'challenge_attempts': '370',
  'retrieve_successes': '54',
  'retrieve_attempts': '74',
  'total_successes': '893',
  'tier': 'Bronze',
  'storage_limit': '1099511627776'},
 '5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2': {'store_attempts'

In [9]:
for k,v in stats.items():
    print(v)

{'store_attempts': '263', 'store_successes': '260', 'challenge_successes': '248', 'challenge_attempts': '351', 'retrieve_successes': '53', 'retrieve_attempts': '77', 'total_successes': '841', 'tier': 'Bronze', 'storage_limit': '1099511627776'}
{'store_attempts': '0', 'store_successes': '0', 'challenge_successes': '0', 'challenge_attempts': '0', 'retrieve_successes': '0', 'retrieve_attempts': '0', 'total_successes': '0', 'tier': 'Bronze', 'storage_limit': '1099511627776'}
{'store_attempts': '273', 'store_successes': '271', 'challenge_successes': '267', 'challenge_attempts': '370', 'retrieve_successes': '54', 'retrieve_attempts': '74', 'total_successes': '893', 'tier': 'Bronze', 'storage_limit': '1099511627776'}
{'store_attempts': '277', 'store_successes': '274', 'challenge_successes': '267', 'challenge_attempts': '364', 'retrieve_successes': '57', 'retrieve_attempts': '78', 'total_successes': '894', 'tier': 'Bronze', 'storage_limit': '1099511627776'}
{'store_attempts': '0', 'store_succe

In [3]:
hotkeys = await active_hotkeys(r)
hotkeys

['5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2',
 '5CaFuijc2ucdoWhkjLaYgnzYrpv62KGt1fWWtUxhFHXPA3KK',
 '5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz',
 '5C82pPChStvLQhLVWxbQSQ52diLvYXxezqP9jhFWSRkHSFon',
 '5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU',
 '5EZCtyfQrDUQqRKYDdq1eaKSB9Ugkr3GxMb8XadewMRMxdev',
 '5EuzvsFVopm2fQ8SLsPsvQdqe4CatZbvrYscxXbrPZsNYmPX',
 '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH',
 '5E4yZGgFSYvMn9gG9U6ciHZgXdPcRxmbX27mbTsz7yRspKiy',
 '5GjBkhRhq5V7yiqdPDFe3k8rj3CsrfW7r6u7S6FKeYJ8627i',
 '5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWsot1xiPAPVVHj1']

In [11]:
hks = await get_hash_keys(hotkeys[0], r)
hks

[b'23399097068315179173129570487705732962442776409318522559127722344604212428089',
 b'20426272407592839195260722477134653049818329866495462607970737373933123255095',
 b'81235527693771513373128871952165240947278114836703145443618693143710405106587',
 b'37126771235288712682231329162643698920095035479717736651200971174687002118205',
 b'97493562267162288680348909099867999961125813605820619357984846205199199003684',
 b'75874901349343040792600229074241222437491736987158447636417385222671857688474',
 b'85472305096896749909146739422967990519857681725588843558554284082640193280693',
 b'12626655446789626172894057401400463234512562893863086047784487342626898098597',
 b'80530548033198852758965270080798200758823723188705669273144334890526368033785',
 b'14123462951905305641953457106303884571834308903902186516228235969476297033082',
 b'86285302215778499394032385558738691895839335723531343775592542779784665578212',
 b'114123709737449883136759727248779931358342524046270912717434002245282173598329',
 b'

In [4]:
caps = await cache_hotkeys_capacity(hotkeys, r)
caps

{'5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2': (335727813,
  1099511627776),
 '5CaFuijc2ucdoWhkjLaYgnzYrpv62KGt1fWWtUxhFHXPA3KK': (324070741,
  1099511627776),
 '5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz': (234376357,
  1099511627776),
 '5C82pPChStvLQhLVWxbQSQ52diLvYXxezqP9jhFWSRkHSFon': (379216706,
  1099511627776),
 '5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU': (246089411,
  1099511627776),
 '5EZCtyfQrDUQqRKYDdq1eaKSB9Ugkr3GxMb8XadewMRMxdev': (303230591,
  1099511627776),
 '5EuzvsFVopm2fQ8SLsPsvQdqe4CatZbvrYscxXbrPZsNYmPX': (233238745,
  1099511627776),
 '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH': (373019605,
  1099511627776),
 '5E4yZGgFSYvMn9gG9U6ciHZgXdPcRxmbX27mbTsz7yRspKiy': (334813387,
  1099511627776),
 '5GjBkhRhq5V7yiqdPDFe3k8rj3CsrfW7r6u7S6FKeYJ8627i': (291950923,
  1099511627776),
 '5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWsot1xiPAPVVHj1': (269442888,
  1099511627776)}

In [15]:
cv_storage = await current_validator_storage(r)
cv_storage

3325177167

In [5]:
tstats = await tier_statistics(database)
tstats

{'counts': {'Super Saiyan': 0,
  'Diamond': 0,
  'Gold': 0,
  'Silver': 0,
  'Bronze': 20},
 'capacity': {'Super Saiyan': 0,
  'Diamond': 0,
  'Gold': 0,
  'Silver': 0,
  'Bronze': 21990232555520},
 'usage': {'Super Saiyan': 0,
  'Diamond': 0,
  'Gold': 0,
  'Silver': 0,
  'Bronze': 3325177167},
 'percent_usage': {'Super Saiyan': 0,
  'Diamond': 0,
  'Gold': 0,
  'Silver': 0,
  'Bronze': 0.015121155079214077}}

In [6]:
istats = await tier_statistics(database, True)
istats

{'Super Saiyan': {'counts': 0, 'capacity': 0, 'usage': 0, 'percent_usage': 0},
 'Diamond': {'counts': 0, 'capacity': 0, 'usage': 0, 'percent_usage': 0},
 'Gold': {'counts': 0, 'capacity': 0, 'usage': 0, 'percent_usage': 0},
 'Silver': {'counts': 0, 'capacity': 0, 'usage': 0, 'percent_usage': 0},
 'Bronze': {'counts': 20,
  'capacity': 21990232555520,
  'usage': 3325177167,
  'percent_usage': 0.015121155079214077}}

In [7]:
successful_requests = await total_successful_requests(r)
successful_requests

9097

In [8]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('network_data.db')
cursor = conn.cursor()

# Create network_stats table
cursor.execute('''
CREATE TABLE IF NOT EXISTS network_stats (
    total_network_capacity_mb INTEGER,
    current_validator_storage_mb INTEGER
)
''')

# Create validator_stats table
cursor.execute('''
CREATE TABLE IF NOT EXISTS validator_stats (
    validator_key TEXT PRIMARY KEY,
    store_attempts INTEGER,
    store_successes INTEGER,
    challenge_successes INTEGER,
    challenge_attempts INTEGER,
    retrieve_successes INTEGER,
    retrieve_attempts INTEGER,
    total_successes INTEGER,
    tier TEXT,
    storage_limit INTEGER
)
''')

conn.commit()


In [9]:
for key, values in (await get_miner_statistics(r)).items():
    cursor.execute('''
    INSERT INTO validator_stats (validator_key, store_attempts, store_successes, challenge_successes, challenge_attempts, retrieve_successes, retrieve_attempts, total_successes, tier, storage_limit)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (key, 
         values['store_attempts'], 
         values['store_successes'], 
         values['challenge_successes'], 
         values['challenge_attempts'], 
         values['retrieve_successes'], 
         values['retrieve_attempts'], 
         values['total_successes'], 
         values['tier'], 
         values['storage_limit']))

conn.commit()

In [10]:
total_network_capacity = await get_network_capacity(r) / (1024 ** 2) # MB
current_storage = await current_validator_storage(r) / (1024 ** 2) # MB

cursor.execute('''
INSERT INTO network_stats (total_network_capacity, current_validator_storage)
VALUES (?, ?)
''', (total_network_capacity, current_storage))

conn.commit()

conn.close()

In [11]:
# Connect to the SQLite database
conn = sqlite3.connect('network_data.db')
cursor = conn.cursor()

# Retrieve and display data from network_stats
cursor.execute('SELECT * FROM network_stats')
network_stats = cursor.fetchall()
print("Network Stats:")
for row in network_stats:
    print(f"Total Network Capacity: {row[0]}, Current Validator Storage: {row[1]}")

# Retrieve and display data from validator_stats
cursor.execute('SELECT * FROM validator_stats')
validator_stats = cursor.fetchall()
print("\nValidator Stats:")
for row in validator_stats:
    print(f"Validator Key: {row[0]}, Store Attempts: {row[1]}, Store Successes: {row[2]}, Challenge Successes: {row[3]}, Challenge Attempts: {row[4]}, Retrieve Successes: {row[5]}, Retrieve Attempts: {row[6]}, Total Successes: {row[7]}, Tier: {row[8]}, Storage Limit: {row[9]}")

# Close the database connection
conn.close()


Network Stats:
Total Network Capacity: 10995249446912, Current Validator Storage: 1014098.6224813461
Total Network Capacity: 10995249446912, Current Validator Storage: 1014098.6224813461

Validator Stats:
Validator Key: 5Cih6V8oc7ahYHAXwnSVu8ys8h3GM4A1kXJqfJFQCwD9XmFy, Store Attempts: 128, Store Successes: 128, Challenge Successes: 73, Challenge Attempts: 78, Retrieve Successes: 7, Retrieve Attempts: 10, Total Successes: 2261, Tier: Gold, Storage Limit: 109951162777600
Validator Key: 5CoEiuGiXZgeFSpccqo17aiovSRh9Tv2Vn6j65J93iEhB2hJ, Store Attempts: 0, Store Successes: 0, Challenge Successes: 0, Challenge Attempts: 81, Retrieve Successes: 0, Retrieve Attempts: 0, Total Successes: 1998, Tier: Bronze, Storage Limit: 1099511627776
Validator Key: 5Do7YsNrKEpejfTMsmaCxh9FPHrxgBi4jYh9VcbeifobjNAe, Store Attempts: 0, Store Successes: 0, Challenge Successes: 0, Challenge Attempts: 0, Retrieve Successes: 0, Retrieve Attempts: 0, Total Successes: 0, Tier: Bronze, Storage Limit: 1099511627776
Vali

In [ ]:
"""
Steps:
- Connect to SQL database on the server
- Create tables for network stats and validator stats
- Insert data into the tables
- Retrieve data from the tables and display it


Stat ideas SQL commands to write:
- Current tier usage vs total capacity (per tier per validator)
- Current miner usgae vs total capacity (per miner UID/Hotkey across all miners)
- Current validator usage vs total capcity (per validator UID/hotkey across all miners)
- % usage of each tier
- % usage of each miner
- % usage of each validator
- Tier counts (how many miners in each tier)
- Tier capacity (how much storage in each tier)
- Tier usage (how much storage is being used in each tier)  
- Total theoretical network capacity (overall across all validators, they have their own copy therefore will have independent caps for each hotkey)
- Current total challenges successes vs total challenge attempts
- Current total store successes vs total store attempts
- Current total retrieve successes vs total retrieve attempts
- Validator (FND) events for each store|challenge|retrieve query round
"""

In [10]:
# Number of data hashes per hotkey
hashes = await get_hashes_for_hotkey(hotkeys[0], database)
print(len(hashes))

54


In [11]:
# Amount of data stored per hotkey (bytes -> megabytes)
hk_storage = await total_hotkey_storage(hotkeys[0], database) / (1024 ** 2)

# Or equivalently for all: (Use this and turn into sql command)
await cache_hotkeys_capacity(hotkeys, database)

{'5FbHff5f2gUqu4r3Kfcm352VAognfCJv2owqCthiyCw8yTi2': (335727813,
  1099511627776),
 '5CaFuijc2ucdoWhkjLaYgnzYrpv62KGt1fWWtUxhFHXPA3KK': (324070741,
  1099511627776),
 '5H3Hc7G9LprdVod3UMHDhzKmTH2dEW2tcooaBMm8K43Bwjiz': (234376357,
  1099511627776),
 '5C82pPChStvLQhLVWxbQSQ52diLvYXxezqP9jhFWSRkHSFon': (379216706,
  1099511627776),
 '5FKwbwguHU1SVQiGot5YKSTQ6fWGVv2wRHFrWfwp9X9nWbyU': (246089411,
  1099511627776),
 '5EZCtyfQrDUQqRKYDdq1eaKSB9Ugkr3GxMb8XadewMRMxdev': (303230591,
  1099511627776),
 '5EuzvsFVopm2fQ8SLsPsvQdqe4CatZbvrYscxXbrPZsNYmPX': (233238745,
  1099511627776),
 '5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH': (373019605,
  1099511627776),
 '5E4yZGgFSYvMn9gG9U6ciHZgXdPcRxmbX27mbTsz7yRspKiy': (334813387,
  1099511627776),
 '5GjBkhRhq5V7yiqdPDFe3k8rj3CsrfW7r6u7S6FKeYJ8627i': (291950923,
  1099511627776),
 '5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWsot1xiPAPVVHj1': (269442888,
  1099511627776)}

In [35]:
# TODO: Delete and redo these tables
# HOTKEYS_TABLE (By Hotkey)
# ID | TIMESTAMP | HOTKEY | TIER | CURRENT_STORAGE | CAPACITY | PERCENT_USAGE | NUM_HASHES | TOTAL_SUCCESSES | STORE_SUCCESSES | STORE_ATTEMPTS | CHALLENGE_SUCCESSES | CHALLENGE_ATTEMPTS | RETRIEVE_SUCCESSES | RETRIEVE_ATTEMPTS | 
# Example to create the HotkeysTable
create_hotkeys_table_sql = '''
CREATE TABLE IF NOT EXISTS HotkeysTable (
    id INT AUTO_INCREMENT PRIMARY KEY,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    hotkey VARCHAR(255) NOT NULL,
    tier VARCHAR(50) NOT NULL,
    current_storage BIGINT NOT NULL,
    capacity BIGINT NOT NULL,
    percent_usage FLOAT NOT NULL,
    num_hashes INT NOT NULL,
    total_successes INT NOT NULL,
    store_successes INT NOT NULL,
    store_attempts INT NOT NULL,
    challenge_successes INT NOT NULL,
    challenge_attempts INT NOT NULL,
    retrieve_successes INT NOT NULL,
    retrieve_attempts INT NOT NULL
)
'''
connect_and_execute(create_hotkeys_table_sql)


# TIER_STATS_TABLE (By Tier)
# ID | TIMESTAMP | TIER | COUNTS | CAPACITY | CURRENT_STORAGE | PERCENT_USAGE | CURRENT_ATTEMPTS | CURRENT_SUCCESSES | GLOBAL_SUCCESS_RATE | TOTAL_GLOBAL_SUCCESSES
create_tier_stats_table = '''
CREATE TABLE IF NOT EXISTS TierStatsTable (
    id INT AUTO_INCREMENT PRIMARY KEY,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    tier VARCHAR(50) NOT NULL,
    counts INT NOT NULL,
    capacity BIGINT NOT NULL,
    current_storage BIGINT NOT NULL,
    percent_usage FLOAT NOT NULL,
    current_attempts INT NOT NULL,
    current_successes INT NOT NULL,
    global_success_rate FLOAT NOT NULL,
    total_global_successes INT NOT NULL
)
'''
connect_and_execute(create_tier_stats_table)


# NETWORK STATS_TABLE (Global)
# ID | TIMESTAMP | CURRENT_STORAGE | NETWORK_CAPACITY | TOTAL_SUCCESSFUL_REQUESTS | REDIS_INDEX_SIZE_MB | GLOBAL_CURRENT_ATTEMPTS | GLOBAL_CURRENT_SUCCESSES | GLOBAL_CURRENT_SUCCESS_RATE
create_network_stats_table = '''
CREATE TABLE IF NOT EXISTS NetworkStatsTable (
    id INT AUTO_INCREMENT PRIMARY KEY,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    current_storage BIGINT NOT NULL,
    network_capacity BIGINT NOT NULL,
    total_successful_requests INT NOT NULL,
    redis_index_size_mb FLOAT NOT NULL,
    global_current_attempts INT NOT NULL,
    global_current_successes INT NOT NULL,
    global_current_success_rate FLOAT NOT NULL
)
'''
connect_and_execute(create_network_stats_table)

Command executed successfully.
Command executed successfully.
Command executed successfully.


In [34]:
def drop_table(name):
    sql = f"DROP TABLE {name};"
    connect_and_execute(sql)

drop_table("TierStatsTable")
drop_table("NetworkStatsTable")
drop_table("HotkeysTable")

Command executed successfully.
Command executed successfully.
Command executed successfully.


In [7]:
# Write the code to fetch/compute the data required for a row of each table.
# ID | TIMESTAMP | HOTKEY | TIER | CURRENT_STORAGE | CAPACITY | PERCENT_USAGE | NUM_HASHES | TOTAL_SUCCESSES | STORE_SUCCESSES | STORE_ATTEMPTS | CHALLENGE_SUCCESSES | CHALLENGE_ATTEMPTS | RETRIEVE_SUCCESSES | RETRIEVE_ATTEMPTS | 

# HOTKEYS_TABLE
stats = await get_miner_statistics(database)

hotkeys = list(stats)

caps = await cache_hotkeys_capacity(hotkeys, database)

for hotkey, stat in stats.items():
    cur, cap = caps[hotkey]
    n_hashes = len( await get_hashes_for_hotkey(hotkey, database))
    row = [hotkey, stat['tier'], cur, cap, cur / cap, n_hashes, stat['total_successes'], stat['store_successes'], stat['store_attempts'], stat['challenge_successes'], stat['challenge_attempts'], stat['retrieve_successes'], stat['retrieve_attempts']]
    print(row)
    sql_insert_command = """
    INSERT INTO HotkeysTable (
        HOTKEY, TIER, CURRENT_STORAGE, CAPACITY, PERCENT_USAGE, NUM_HASHES, TOTAL_SUCCESSES, STORE_SUCCESSES, STORE_ATTEMPTS, CHALLENGE_SUCCESSES, CHALLENGE_ATTEMPTS, RETRIEVE_SUCCESSES, RETRIEVE_ATTEMPTS
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    connect_and_execute_params(sql_insert_command, row)

['5CVSjN2fMU7mP4B7oAG1V6LsNqyQNLLhDWsot1xiPAPVVHj1', 'Bronze', 269442888, 1099511627776, 0.0002450568790663965, 56, '866', '267', '270', '254', '336', '51', '69']
Command executed successfully.
['5C86aJ2uQawR6P6veaJQXNK9HaWh6NMbUhTiLs65kq4ZW3NH', 'Bronze', 373019605, 1099511627776, 0.0003392593544049305, 60, '851', '259', '262', '257', '377', '54', '81']
Command executed successfully.
['5HapVLypxEsZWvfwd12jUXdp9W1pXAoSbQVzirC6AUDNYBii', 'Bronze', 0, 1099511627776, 0.0, 0, '0', '0', '0', '0', '0', '0', '0']
Command executed successfully.
['5Gpjcz44ViQJzjA14xAgMSZwTLeULeJdXhjKMV6JM3XrGkUw', 'Bronze', 0, 1099511627776, 0.0, 0, '42', '0', '0', '0', '0', '0', '0']
Command executed successfully.
['5CDTTsvLNyrKX9W9DF2CsvR7jpXaue8SUSntPcem1Psq9eb9', 'Bronze', 0, 1099511627776, 0.0, 0, '0', '0', '0', '0', '0', '0', '0']
Command executed successfully.
['5H1CaMTf6XvzSYLcLe4AhnXwttQ4z3ry6fy9XzS9wrxV5vFA', 'Bronze', 0, 1099511627776, 0.0, 0, '251', '47', '51', '37', '67', '9', '9']
Command executed

In [8]:
# Write the code to get the data prepared for TierStatsTable
# ID | TIMESTAMP | TIER | COUNTS | CAPACITY | CURRENT_STORAGE | PERCENT_USAGE | CURRENT_ATTEMPTS | CURRENT_SUCCESSES | GLOBAL_SUCCESS_RATE | TOTAL_GLOBAL_SUCCESSES

tstats = await tier_statistics(database)

istats = {}
for category, tier_dict in tstats.items():
    for tier, value in tier_dict.items():
        if tier not in istats:
            istats[tier] = {}
        istats[tier][category] = value

by_tier = await compute_by_tier_stats(database)

rows = []
for tier, stat in istats.items():
    print(tier, stat)
    row = [tier] + list(stat.values())
    if tier in by_tier:
        tr = by_tier[tier]
        row += [tr['total_current_attempts'], tr['total_current_successes'], tr['success_rate'], tr['total_global_successes']]
    else:
        row += [0, 0, 0, 0]
    print(row)

# Write the actual row to the table
sql_insert_command = """
INSERT INTO TierStatsTable (
    TIER, COUNTS, CAPACITY, CURRENT_STORAGE, PERCENT_USAGE, CURRENT_ATTEMPTS, CURRENT_SUCCESSES, GLOBAL_SUCCESS_RATE, TOTAL_GLOBAL_SUCCESSES
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
connect_and_execute_params(sql_insert_command, row)

Super Saiyan {'counts': 0, 'capacity': 0, 'usage': 0, 'percent_usage': 0}
['Super Saiyan', 0, 0, 0, 0, 0, 0, 0, 0]
Diamond {'counts': 0, 'capacity': 0, 'usage': 0, 'percent_usage': 0}
['Diamond', 0, 0, 0, 0, 0, 0, 0, 0]
Gold {'counts': 0, 'capacity': 0, 'usage': 0, 'percent_usage': 0}
['Gold', 0, 0, 0, 0, 0, 0, 0, 0]
Silver {'counts': 0, 'capacity': 0, 'usage': 0, 'percent_usage': 0}
['Silver', 0, 0, 0, 0, 0, 0, 0, 0]
Bronze {'counts': 20, 'capacity': 21990232555520, 'usage': 3325177167, 'percent_usage': 0.015121155079214077}
['Bronze', 20, 21990232555520, 3325177167, 0.015121155079214077, 7272, 6011, 82.65951595159517, 9097]
Command executed successfully.


In [10]:
# Write the code to get data prepped for NetworkStatsTable
# ID | TIMESTAMP | CURRENT_STORAGE | NETWORK_CAPACITY | TOTAL_SUCCESSFUL_REQUESTS | REDIS_INDEX_SIZE_MB | GLOBAL_CURRENT_ATTEMPTS | GLOBAL_CURRENT_SUCCESSES | GLOBAL_CURRENT_SUCCESS_RATE
net_cap = await get_network_capacity(database)

idx_size = await get_redis_db_size(database)

tot_suc = await total_successful_requests(database)

hotkeys = await active_hotkeys(database)
caps = await cache_hotkeys_capacity(hotkeys, database)
cur_storage = sum(list(zip(*list(caps.values())))[0])

store_attempts = sa = 0
store_successes = ss = 0
challenge_attempts = ca = 0
challenge_successes = cs = 0
retrieve_attempts = ra = 0
retrieve_successes = rs = 0

for _, d in (await get_miner_statistics(database)).items():
    tier = d['tier']
    sa += int(d['store_attempts'])
    ss += int(d['store_successes'])
    ca += int(d['challenge_attempts'])
    cs += int(d['challenge_successes'])
    ra += int(d['retrieve_attempts'])
    rs += int(d['retrieve_successes'])

cta = sum([sa, ca, ra])
cts = sum([ss, cs, rs])
print(cts, cta)
print(cts / cta, "%")
global_attempts = cta
global_successees = cts

row = [cur_storage, net_cap, tot_suc, idx_size, global_attempts, global_successees, global_successees / global_attempts]
print(row)
# Write SQL to populate table with this row
# TODO: FIX THIS!
sql_insert_command = """
INSERT INTO NetworkStatsTable (
    CURRENT_STORAGE, NETWORK_CAPACITY, TOTAL_SUCCESSFUL_REQUESTS, REDIS_INDEX_SIZE_MB, GLOBAL_CURRENT_ATTEMPTS, GLOBAL_CURRENT_SUCCESSES, GLOBAL_CURRENT_SUCCESS_RATE
) VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

# Assuming 'row' is already defined as shown in your example
connect_and_execute_params(sql_insert_command, row)


6011 7272
0.8265951595159516 %
[3325177167, 21990232555520, 9097, 280791, 7272, 6011, 0.8265951595159516]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:47                                                                                   │
│                                                                                                  │
│   44 """                                                                                         │
│   45                                                                                             │
│   46 # Assuming 'row' is already defined as shown in your example                                │
│ ❱ 47 connect_and_execute(sql_insert_command, row)                                                │
│   48                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: connect_and_execute() takes 1 positional argument but 2 were given

In [40]:
# Write the SQL to retrieve the latest row of each hotkey:

sql = """
SELECT h.*
FROM HotkeysTable h
INNER JOIN (
    SELECT HOTKEY, MAX(TIMESTAMP) as LatestTimestamp
    FROM HotkeysTable
    GROUP BY HOTKEY
) AS latest ON h.HOTKEY = latest.HOTKEY AND h.TIMESTAMP = latest.LatestTimestamp;
"""

connect_and_execute(sql)

Command executed successfully.


In [4]:
def connect_and_execute(command):
    import pymysql

    db_params = {
        'host': 'localhost',
        'user': 'filetao',
        'password': 'filetao',
        'database': 'sn21',
        'charset': 'utf8mb4',
        'cursorclass': pymysql.cursors.DictCursor
    }

    try:
        # Connect to the database
        conn = pymysql.connect(**db_params)
        with conn.cursor() as cursor:
            # Execute the SQL command
            cursor.execute(command)
            # Commit the transaction
            conn.commit()
            print("Command executed successfully.")
    except pymysql.MySQLError as e:
        print(f"MySQL Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Ensure the connection is closed
        if 'conn' in locals() or 'conn' in globals():
            conn.close()

In [5]:
def connect_and_execute_params(sql_command, params=None):
    import pymysql

    db_params = {
        'host': 'localhost',
        'user': 'filetao',
        'password': 'filetao',
        'database': 'sn21',
        'charset': 'utf8mb4',
        'cursorclass': pymysql.cursors.DictCursor
    }

    try:
        # Connect to the database
        conn = pymysql.connect(**db_params)
        with conn.cursor() as cursor:
            # Execute the SQL command with parameters
            cursor.execute(sql_command, params)
            # Commit the transaction
            conn.commit()
            print("Command executed successfully.")
    except pymysql.MySQLError as e:
        print(f"MySQL Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Ensure the connection is closed
        if 'conn' in locals() or 'conn' in globals():
            conn.close()

"""
# Example usage:
sql = "INSERT INTO YourTable (column1, column2) VALUES (%s, %s)"
params = ('value1', 'value2')
connect_and_execute(sql, params)
"""

'\n# Example usage:\nsql = "INSERT INTO YourTable (column1, column2) VALUES (%s, %s)"\nparams = (\'value1\', \'value2\')\nconnect_and_execute(sql, params)\n'

In [6]:
def connect_and_fetch(sql_command, params=None):
    import pymysql

    db_params = {
        'host': 'localhost',
        'user': 'filetao',
        'password': 'filetao',
        'database': 'sn21',
        'charset': 'utf8mb4',
        'cursorclass': pymysql.cursors.DictCursor
    }

    try:
        # Connect to the database
        conn = pymysql.connect(**db_params)
        with conn.cursor() as cursor:
            # Execute the SQL command
            cursor.execute(sql_command, params)
            
            # Fetch the results
            results = cursor.fetchall()
            
            # If you want to print the results:
            for row in results:
                print(row)
            
            # Or if you prefer to return the results for further processing:
            return results
            
    except pymysql.MySQLError as e:
        print(f"MySQL Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Ensure the connection is closed
        if 'conn' in locals() or 'conn' in globals():
            conn.close()

'''
# SQL query to retrieve the latest row of each hotkey
sql = """
SELECT h.*
FROM HotkeysTable h
INNER JOIN (
    SELECT HOTKEY, MAX(TIMESTAMP) as LatestTimestamp
    FROM HotkeysTable
    GROUP BY HOTKEY
) AS latest ON h.HOTKEY = latest.HOTKEY AND h.TIMESTAMP = latest.LatestTimestamp;
"""

# Execute the command and print or process the fetched data
fetched_data = connect_and_fetch(sql)
'''

'\n# SQL query to retrieve the latest row of each hotkey\nsql = """\nSELECT h.*\nFROM HotkeysTable h\nINNER JOIN (\n    SELECT HOTKEY, MAX(TIMESTAMP) as LatestTimestamp\n    FROM HotkeysTable\n    GROUP BY HOTKEY\n) AS latest ON h.HOTKEY = latest.HOTKEY AND h.TIMESTAMP = latest.LatestTimestamp;\n"""\n\n# Execute the command and print or process the fetched data\nfetched_data = connect_and_fetch(sql)\n'